<a href="https://colab.research.google.com/github/gfgullo/NeuralDante/blob/main/neural_dante.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Neural Dante

## Scarichiamo il dataset

In [ ]:
!wget https://dmf.unicatt.it/~della/pythoncourse18/commedia.txt

--2023-04-29 16:42:05--  https://dmf.unicatt.it/~della/pythoncourse18/commedia.txt
Resolving dmf.unicatt.it (dmf.unicatt.it)... 185.11.152.34
Connecting to dmf.unicatt.it (dmf.unicatt.it)|185.11.152.34|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 557962 (545K) [text/plain]
Saving to: ‘commedia.txt.10’

commedia.txt.10     100%[===================>] 544.88K   698KB/s    in 0.8s    

2023-04-29 16:42:07 (698 KB/s) - ‘commedia.txt.10’ saved [557962/557962]



## Importiamo i moduli

In [ ]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"

In [ ]:
import re
import spacy
import sklearn
import numpy as np
from random import randint
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras import Model, Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.callbacks import LambdaCallback
from tensorflow.keras.utils import to_categorical

In [ ]:
!python -m spacy download it_core_news_sm

2023-04-29 16:46:47.622139: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.0/13.0 MB 91.8 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('it_core_news_sm')


In [ ]:
nlp = spacy.load("it_core_news_sm")

## Preprocessiamo il testo

In [ ]:
f = open("commedia.txt", "r", encoding="utf-8")
commedia = f.read()
commedia[:100]

'LA DIVINA COMMEDIA\ndi Dante Alighieri\nINFERNO\n\n\n\nInferno: Canto I\n\n  Nel mezzo del cammin di nostra '

In [ ]:
def preprocessing_text(text):
  text = text.lower()
  text = text.replace("'"," ").replace("\n"," ")
  text = re.sub(r'[^\w\s]', '', text)
  text = re.sub(' +', ' ', text)
  return text

In [ ]:
commedia = preprocessing_text(commedia)
commedia[:500]

'la divina commedia di dante alighieri inferno inferno canto i nel mezzo del cammin di nostra vita mi ritrovai per una selva oscura ché la diritta via era smarrita ahi quanto a dir qual era è cosa dura esta selva selvaggia e aspra e forte che nel pensier rinova la paura tant è amara che poco è più morte ma per trattar del ben ch i vi trovai dirò de l altre cose ch i v ho scorte io non so ben ridir com i v intrai tant era pien di sonno a quel punto che la verace via abbandonai ma poi ch i fui al p'

In [ ]:
def tokenize(text):
  tokens = nlp(text)
  tokens_filtered = [token.text for token in tokens]
  return tokens_filtered

tokens = tokenize(commedia)

In [ ]:
tokens[:10]

['la',
 'divina',
 'commedia',
 'di',
 'dante',
 'alighieri',
 'inferno',
 'inferno',
 'canto',
 'i']

In [ ]:
maxlen = 11

sents = []

for i in range(maxlen, len(tokens)):
  sents.append(tokens[i-maxlen:i])

In [ ]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(sents)
sents = tokenizer.texts_to_sequences(sents)
sents[0]

[4, 350, 3582, 6, 2648, 3581, 248, 248, 106, 26, 41]

In [ ]:
sents = np.array(sents)
X = sents[:,:-1]
y = sents[:,-1]

In [ ]:
vocab_size = len(tokenizer.word_docs)

In [ ]:
y = to_categorical(y)
y.shape

(101901, 12822)

## Creazione della rete ricorrente

In [ ]:
model = Sequential()
model.add(Embedding(vocab_size+1, maxlen-1, input_length=maxlen-1))
model.add(LSTM(50, return_sequences=True))
model.add(LSTM(50))
model.add(Dense(50, activation="relu"))
model.add(Dense(vocab_size+1, activation="softmax"))

In [ ]:
model.compile(loss="categorical_crossentropy", optimizer="adam", metrics="accuracy")

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 10, 10)            128220    
                                                                 
 lstm (LSTM)                 (None, 10, 50)            12200     
                                                                 
 lstm_1 (LSTM)               (None, 50)                20200     
                                                                 
 dense (Dense)               (None, 50)                2550      
                                                                 
 dense_1 (Dense)             (None, 12822)             653922    
                                                                 
Total params: 817,092
Trainable params: 817,092
Non-trainable params: 0
_________________________________________________________________


In [ ]:
def generate(seed=None, random_seed_length=10, generate_len=25):

  output = ""

  if seed is None:
    start_index = randint(0, len(commedia))
    text = commedia[start_index:start_index+random_seed_length]
  else:
    text = preprocessing_text(seed)

  for i in range(generate_len):

    tokens = np.array(tokenizer.texts_to_sequences([text]))
    tokens = pad_sequences(tokens, maxlen=maxlen-1)

    pred_token = np.argmax(model.predict([tokens], verbose=False), axis=1)[0]
    pred_word = tokenizer.index_word[pred_token+1]

    text+=" "+pred_word
    output+=pred_word+" "

  return output

In [ ]:
def generate_on_epoch(epoch, _):
  output = generate()
  print('\nDante dice: "'+output+'"')

In [ ]:
epoch_callback = LambdaCallback(on_epoch_end=generate_on_epoch)
model.fit(X, y, batch_size=128, epochs=100, callbacks=[epoch_callback])

Epoch 1/100
797/797 [==============================] - ETA: 0s - loss: 7.2586 - accuracy: 0.0391
Dante dice: "che che che che che che che che che che che che che che che che che che che che che che che che che "
797/797 [==============================] - 34s 34ms/step - loss: 7.2586 - accuracy: 0.0391
Epoch 2/100
797/797 [==============================] - ETA: 0s - loss: 6.8953 - accuracy: 0.0398
Dante dice: "che che che che che che che che che che che che che che che che che che che che che che che che che "
797/797 [==============================] - 14s 18ms/step - loss: 6.8953 - accuracy: 0.0398
Epoch 3/100
797/797 [==============================] - ETA: 0s - loss: 6.7530 - accuracy: 0.0454
Dante dice: "che la ben che l ben che l ben che l ben che l ben che l ben che l ben che l ben che "
797/797 [==============================] - 13s 16ms/step - loss: 6.7530 - accuracy: 0.0454
Epoch 4/100
797/797 [==============================] - ETA: 0s - loss: 6.6446 - accuracy: 0.0518
Dante dic

In [ ]:
text = "ciao come stai"

tokens = np.array(tokenizer.texts_to_sequences([text]))
tokens = pad_sequences(tokens, maxlen=maxlen-1)

pred_proba = model.predict([tokens], verbose=False)
pred_token = np.argmax(pred_proba, axis=1)[0]
print(pred_proba)
pred_word = tokenizer.index_word[pred_token]
